0- lat <br>1- lon<br> 2- alt 

# CONFIDENCE 3 

In [1]:
from IPython.display import HTML
import random

def hide(for_next=False):
    this_cell = """$('div.cell.code_cell.rendered.selected')"""
    next_cell = this_cell + '.next()'

    toggle_text = 'FUNCTIONAL COMPONENT HIDDEN'  # text shown on toggle link
    target_cell = this_cell  # target cell to control with toggle
    js_hide_current = ''  # bit of JS to permanently hide code in current cell (only when toggling next cell)

    if for_next:
        target_cell = next_cell
        toggle_text += ' next cell'
        js_hide_current = this_cell + '.find("div.input").hide();'

    js_f_name = 'code_toggle_{}'.format(str(random.randint(1,2**64)))

    html = """
        <script>
            function {f_name}() {{
                {cell_selector}.find('div.input').toggle();
            }}

            {js_hide_current}
        </script>

        <a href="javascript:{f_name}()">{toggle_text}</a>
    """.format(
        f_name=js_f_name,
        cell_selector=target_cell,
        js_hide_current=js_hide_current, 
        toggle_text=toggle_text
    )

    return HTML(html)

hide()

In [2]:
# IMPORTING LIBRARIES 
import math
import numpy as np
from math import radians, cos, sin, asin, sqrt, atan
import geopy
from geopy import distance as GD
from numpy import cross, eye, dot
from scipy.linalg import expm, norm
from yattag import Doc, indent

In [3]:
# FUNCTIONS


def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    
    
    bearing = math.atan2(sin(lon2-lon1)*cos(lat2), cos(lat1)*sin(lat2)-sin(lat1)*cos(lat2)*cos(lon2-lon1))
    bearing = math.degrees(bearing)
    bearing = (bearing + 360) % 360

    
    
    return c * r , bearing


def dot(vA, vB):
    return vA[0]*vB[0]+vA[1]*vB[1]
def ang(vA, vB):
    # Get dot prod
    dot_prod = dot(vA, vB)
    # Get magnitudes
    magA = dot(vA, vA)**0.5
    magB = dot(vB, vB)**0.5
    # Get cosine value
    cos_ = dot_prod/magA/magB
    # Get angle in radians and then convert to degrees
    angle = math.acos(dot_prod/magB/magA)
    # Basically doing angle <- angle mod 360
    ang_deg = math.degrees(angle)%360

    if ang_deg-180>=0:
        # As in if statement
        return 360 - ang_deg
    else: 

        return ang_deg

def get_distance(x,y):
    return math.sqrt((x**2+y**2))/1000

import geopy
def get_coord_2(bearing,lat,lon,dist):
    b = bearing
    d = dist
    lat1 = lat 
    lon1 = lon
    origin = geopy.Point(lat1, lon1)
    for model_name in models :
        destination = GD.geodesic(kilometers=d,ellipsoid=model_name).destination(origin, b,)
        lat2, lon2 = destination.latitude, destination.longitude
        print(model_name,' \t: ',lat2,lon2)

def get_coord_3(bearing,lat,lon,dist):
#     COPY OF FUNCTION 2  but fixed at the GRS-67 
    b = bearing
    d = dist
    lat1 = lat 
    lon1 = lon
    origin = geopy.Point(lat1, lon1)
    

    destination = GD.geodesic(kilometers=d,ellipsoid='GRS-67').destination(origin, b,)
    lat2, lon2 = destination.latitude, destination.longitude
    return(lat2,lon2)
        
        
ELLIPSOIDS = {'WGS-84':        (6378.137,    6356.7523142,  1 / 298.257223563),
              'GRS-80':        (6378.137,    6356.7523141,  1 / 298.257222101),
              'Airy (1830)':   (6377.563396, 6356.256909,   1 / 299.3249646),
              'Intl 1924':     (6378.388,    6356.911946,   1 / 297.0),
              'Clarke (1880)': (6378.249145, 6356.51486955, 1 / 293.465),
              'GRS-67':        (6378.1600,   6356.774719,   1 / 298.25),
              }

models = list(ELLIPSOIDS.keys())

def get_coord_1(bearing,lat,lon,dist,Rmodel='dji'):
    brng = math.radians(bearing)
    
    d = dist 
    if Rmodel=='dji' :
        R = 6378.137
    elif Rmodel=='google':
        R = 6371.0710
    lat1 = math.radians(lat)
    lon1 = math.radians(lon) 
    lat2 = math.asin( math.sin(lat1)*math.cos(d/R) + math.cos(lat1)*math.sin(d/R)*math.cos(brng))
    lon2 = lon1 + math.atan2(math.sin(brng)*math.sin(d/R)*math.cos(lat1),math.cos(d/R)-math.sin(lat1)*math.sin(lat2))
    lat2 = math.degrees(lat2)
    lon2 = math.degrees(lon2)
    
    print (Rmodel,' \t\t', lat2,lon2)

hide()

In [4]:
# INPUTS 

# GPS Coordinates of the lidar 
lat0 = 11.12330329
lon0 = 77.32846783

# GPS Coordinate of known point  # Point 1 - RTK 
lat1 = 11.123292860
lon1 = 77.328536142 
 

# Lidar Drone Offset distance between lidar and drone, height wise 
lidar_drone_offset = 1
   
# Q Elevation of lidar from ground 
theta = math.radians(-19.2)
tY = np.array([[math.cos(theta),0,math.sin(theta),0],
          [0,1,0,0],
          [-1*math.sin(theta),0,math.cos(theta),0],
          [0,0,0,1]
         ])


# Point 1 ->  Coordinates (x,y) INITIAL CALIBRATION POINT - RTK 
vA = [6.906466216,2.549277027,0.2232162162]


PLUSARR1 = [] 
PLUSARR2 = []
PLUSARR3 = []

NEGARR1 = []
NEGARR2 = []
NEGARR3 = []



REF_PLUSARR1 = [] 
REF_PLUSARR2 = [] 
REF_PLUSARR3 = [] 

REF_NEGARR1 = []
REF_NEGARR2 = []
REF_NEGARR3 = []


In [5]:
ARR1 = [[22,33,44],[22,33,33],[33,39,298],[33,39,298],[33,39,298]]   # Picture Array - Left
ARR2 = [[22,33,44],[22,33,33],[33,39,298],[33,39,298],[33,39,298]]   # Picture Array - Centre
ARR3 = [[22,33,44],[22,33,33],[33,39,298],[33,39,298],[33,39,298]]   # Picture Array - Right
REF = [[22,33,44],[22,33,33],[33,39,298],[33,39,298],[33,39,298]]   # Reference Array 

### CALCULATION

In [6]:
distacePointRtk, bearing = haversine(lon0, lat0, lon1, lat1)

In [7]:
for i in ARR1 :
    # LONGLAT 
    vB = i
    x = vB[0]
    y = vB[1]
    offset = ang(vA,vB) # Offset of p1 from p2 
    
    # Options 
    bPlus = bearing + offset
    bNeg = bearing - offset
    
    latPlus,lonPlus = get_coord_3(bearing=bPlus,lat=lat0,lon=lon0,dist=get_distance(x,y))
    latNeg,lonNeg = get_coord_3(bearing=bNeg,lat=lat0,lon=lon0,dist=get_distance(x,y))
    
    
    
    # ALTITUDE 
    target = np.array([[vB[0]],[vB[1]],[vB[2]],[1]])
    res = tY @ target
    height = res[2]
    height = height - lidar_drone_offset

    
    PLUSARR1.append([latPlus,lonPlus,height[0]])
    NEGARR1.append([latNeg,latNeg,height[0]])

for i in ARR2 :
    # LONGLAT 
    vB = i
    x = vB[0]
    y = vB[1]
    offset = ang(vA,vB) # Offset of p1 from p2 
    
    # Options 
    bPlus = bearing + offset
    bNeg = bearing - offset
    
    latPlus,lonPlus = get_coord_3(bearing=bPlus,lat=lat0,lon=lon0,dist=get_distance(x,y))
    latNeg,lonNeg = get_coord_3(bearing=bNeg,lat=lat0,lon=lon0,dist=get_distance(x,y))
    
    
    
    # ALTITUDE 
    target = np.array([[vB[0]],[vB[1]],[vB[2]],[1]])
    res = tY @ target
    height = res[2]
    height = height - lidar_drone_offset

    
    PLUSARR2.append([latPlus,lonPlus,height[0]])
    NEGARR2.append([latNeg,latNeg,height[0]])

for i in ARR3 :
    # LONGLAT 
    vB = i
    x = vB[0]
    y = vB[1]
    offset = ang(vA,vB) # Offset of p1 from p2 
    
    # Options 
    bPlus = bearing + offset
    bNeg = bearing - offset
    
    latPlus,lonPlus = get_coord_3(bearing=bPlus,lat=lat0,lon=lon0,dist=get_distance(x,y))
    latNeg,lonNeg = get_coord_3(bearing=bNeg,lat=lat0,lon=lon0,dist=get_distance(x,y))
    
    
    
    # ALTITUDE 
    target = np.array([[vB[0]],[vB[1]],[vB[2]],[1]])
    res = tY @ target
    height = res[2]
    height = height - lidar_drone_offset

    
    PLUSARR3.append([latPlus,lonPlus,height[0]])
    NEGARR3.append([latNeg,latNeg,height[0]])



In [8]:
for i in range(len(REF)) :
    vB = REF[i]
    x = vB[0]
    y = vB[1]
    offset = ang(vA,vB) # Offset of p1 from p2 
    
    # Options 
    bPlus = bearing + offset
    bNeg = bearing - offset
    
    latPlus,lonPlus = get_coord_3(bearing=bPlus,lat=lat0,lon=lon0,dist=get_distance(x,y))
    latNeg,lonNeg = get_coord_3(bearing=bNeg,lat=lat0,lon=lon0,dist=get_distance(x,y))
    

    
    p = haversine(lonPlus, latPlus, PLUSARR1[i][1], PLUSARR1[i][0])
    n = haversine(lonNeg, latNeg, NEGARR1[i][1], NEGARR1[i][0])
    
    REF_PLUSARR1.append(p[1])
    REF_NEGARR1.append(n[1])

    
for i in range(len(REF)) :
    vB = REF[i]
    x = vB[0]
    y = vB[1]
    offset = ang(vA,vB) # Offset of p1 from p2 
    
    # Options 
    bPlus = bearing + offset
    bNeg = bearing - offset
    
    latPlus,lonPlus = get_coord_3(bearing=bPlus,lat=lat0,lon=lon0,dist=get_distance(x,y))
    latNeg,lonNeg = get_coord_3(bearing=bNeg,lat=lat0,lon=lon0,dist=get_distance(x,y))
    

    
    p = haversine(lonPlus, latPlus, PLUSARR2[i][1], PLUSARR2[i][0])
    n = haversine(lonNeg, latNeg, NEGARR2[i][1], NEGARR2[i][0])
    
    REF_PLUSARR2.append(p[1])
    REF_NEGARR2.append(n[1])

for i in range(len(REF)) :
    vB = REF[i]
    x = vB[0]
    y = vB[1]
    offset = ang(vA,vB) # Offset of p1 from p2 
    
    # Options 
    bPlus = bearing + offset
    bNeg = bearing - offset
    
    latPlus,lonPlus = get_coord_3(bearing=bPlus,lat=lat0,lon=lon0,dist=get_distance(x,y))
    latNeg,lonNeg = get_coord_3(bearing=bNeg,lat=lat0,lon=lon0,dist=get_distance(x,y))
    

    
    p = haversine(lonPlus, latPlus, PLUSARR3[i][1], PLUSARR3[i][0])
    n = haversine(lonNeg, latNeg, NEGARR3[i][1], NEGARR3[i][0])
    
    REF_PLUSARR3.append(p[1])
    REF_NEGARR3.append(n[1])
  

In [9]:
def get_KML(p1,p2,p3,p4,target):
    
    if LOOKATS = -1 :
        heading = 0 
    
    index = 0 
    
    

    doc, tag, text = Doc().tagtext()
    
    with tag("kml",('xmlns',"http://www.opengis.net/kml/2.2")):
        with tag("Document",('xmlns',"")):
            with tag("name"):
                text('C2')
            with tag('open'):
                text("1")
            with tag("ExtendedData",('xmlns:mis','www.dji.com')):
                with tag('mis:type'):
                    text('Waypoint')
                with tag('mis:stationType'):     # CHECK THIS SETTING AND WHAT IT IS 
                    text('0')
            with tag('Style',('id','waypointStyle')):
                with tag('IconStyle'):
                    with tag('Icon'):
                        with tag('href'):
                            text('https://cdnen.dji-flighthub.com/static/app/images/point.png')

                        
            trig = 1
            for pathar in target: 

                if trig == 1 :
                    trig= trig*-1
                    ref_new = ref
                    pathar = pathar[::-1]
                    ref_new = ref[::-1]
                elif trig == -1:
                    trig= trig*-1
                    ref_new = ref 


                for i in range(len(pathar)):
                    d,b = haversine(  pathar[i][1],pathar[i][0],ref_new[i][1],ref_new[i][0]  )
                    a = pathar[i][2]

        

                
                if headingvar !=0:
                    headingvar = REFARRAY[i]
                    
                with tag('Folder'):
                    with tag('Placemark'):
                        with tag('name'):
                            text(f'Waypoint{i+1}')
                        with tag('visibility'):
                            text(f'1')
                        with tag('description'):
                            text(f'Waypoint')
                        with tag('styleUrl'):
                            text(f'#waypointStyle')

                        with tag('ExtendedData',('xmlns:mis', "www.dji.com")):
                            with tag('mis:useWaylineAltitude'):
                                text('false')

                            with tag('mis:heading'):
                                text(f'{int(headingvar)}')

                            with tag('mis:turnMode'):
                                text('Auto')

                            with tag('mis:gimbalPitch'):
                                text('0.0')

                            with tag('mis:useWaylineSpeed'):
                                text('false')

                            with tag('mis:speed'):
                                text('1.0')

                            with tag('mis:useWaylineHeadingMode'):
                                text('true')

                            with tag('mis:useWaylinePointType'):
                                text('true')

                            with tag('mis:pointType'):
                                text('LineStop')

                            with tag('mis:cornerRadius'):
                                text('0.2')

                            with tag('mis:actions',('param', "10000"),('accuracy', "0"),('cameraIndex', "0"),('payloadType', "0")
                                     ,('payloadIndex', "0")):
                                text('Hover')

                        with tag("Point"):
                            with tag('altitudeMode'):
                                    text('relativeToGround')

                            with tag('coordinates'):
                                    text(f'{long},{lat},{alt}')

            with tag('Placemark'):
                with tag('name'):
                    text(f'Wayline')

                with tag('description'):
                    text(f'Wayline')

                with tag('visibility'):
                    text(f'1')

                with tag('ExtendedData',('xmlns:mis', "www.dji.com")):

                    with tag('mis:altitude'):
                        text('20.0')

                    with tag('mis:autoFlightSpeed'):
                        text(f'1.0')

                    with tag('mis:actionOnFinish'):
                        text('Hover')

                    with tag('mis:headingMode'):
                        text('UsePointSetting')

                    with tag('mis:gimbalPitchMode'):
                        text('UsePointSetting')

                    with tag('mis:powerSaveMode'):
                        text('false')

                    with tag('mis:waypointType'):
                        text('LineStop')

                    with tag('mis:droneInfo'):
                        with tag('mis:droneType'):
                            text('COMMON')
                        with tag('mis:advanceSettings'):
                            text('false')
                        with tag('mis:droneCameras'):
                            text('')
                        with tag('mis:droneHeight'):
                            with tag('mis:useAbsolute'):
                                text('false')
                            with tag('mis:hasTakeoffHeight'):
                                text('false')
                            with tag('mis:takeoffHeight'):
                                text('0.0')


                with tag('styleUrl'):
                    text('#waylineGreenPoly')

                with tag("LineString"):
                    with tag('tessellate'):
                            text('1')

                    with tag('altitudeMode'):
                            text('relativeToGround')

                    with tag('coordinates'):
                        coordinates_total = ''

                        for i in ARRAY:
                            coordinates_total = coordinates_total + str(i[1]) + ',' + str(i[0]) + ',' + str(i[2])[:5] + ' '

                        text(f'{coordinates_total}')


    result = indent(
        doc.getvalue(),
        indentation = ' '*4,
        newline = '\r\n'
    )

    

    save_path_file = filename
    with open(save_path_file, "w") as f: 
        f.write(result)  




In [10]:
get_KML(NEGARR1,REF_NEGARR1,'neg1.kml')

In [9]:
target = [
    [[11.181466337548743,77.29612865573857,-0.4105842622208552],
    [11.181466208757243,77.29612860132971,0.07141573777914478]],
    
    [[11.181446283249944,77.29605570269041,-0.5557416977335411],
    [11.181446192151906,77.29605561810162,-0.07374169773354117]],
    
   [ [11.181397218747597,77.29607240088858,-0.7734157377791463],
    [11.181397083784363,77.29607233321099,-0.2914157377791463]],
    
  [  [11.181416031671274,77.296164329097,-0.6282583022664604],
    [11.181415871414679,77.29616433365368,-0.14625830226646042]],
    
]

ref =    [[11.181430990816661,77.29610239092531,-0.5920000000000007],
    [11.18143085706144,77.29610232910755,-0.11000000000000079]]


In [38]:
import math
import numpy as np
from math import radians, cos, sin, asin, sqrt, atan

from numpy import cross, eye, dot



def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    
    
    bearing = math.atan2(sin(lon2-lon1)*cos(lat2), cos(lat1)*sin(lat2)-sin(lat1)*cos(lat2)*cos(lon2-lon1))
    bearing = math.degrees(bearing)
    bearing = (bearing + 360) % 360
    return c * r , bearing

216.09431244658666 0.07141573777914478
216.09044972167598 -0.4105842622208552
108.46319839853106 -0.5557416977335411
108.5027853427273 -0.07374169773354117
41.06550065875649 -0.2914157377791463
41.060971419471684 -0.7734157377791463
283.83064016573843 -0.6282583022664604
283.8399390029872 -0.14625830226646042


(0.006965955421758946, 283.8399390029872)